# Extract spectrogram images from wave files

- expects the chords to be stored in .wav files in `chords/maj` and `chords/min`
- each .wav file should contain exactly one chord and be at least 3 seconds long
- spectrograms will be saves as .png files and stored into `spectros/maj` and `spectros/min`

In [5]:
import os
import librosa
import numpy as np
from PIL import Image

def load_chord(input_path, output_folder, shapes_set):
    output_filename = os.path.basename(input_path) + '.png'
    output_path = os.path.join(output_folder, output_filename)  

    os.makedirs(output_folder, exist_ok=True)

    [y, _] = librosa.load(input_path, sr=16000)
    max_length = 16000 * 3
    y = librosa.util.fix_length(y, size=max_length)
    spectrogram = librosa.stft(y)
    amplitudes_db = librosa.amplitude_to_db(np.abs(spectrogram), ref=np.max)
    normalized = (amplitudes_db - amplitudes_db.min()) / (amplitudes_db.max() - amplitudes_db.min())
    as_image = (normalized * 255).astype(np.uint8)
    shapes_set.add(as_image.shape)
    image = Image.fromarray(as_image)
    image.save(output_path)


def process_chords(input_dir, output_dir):
    unique_shapes = set()
    for root, _, files in os.walk(input_dir):
        for file in files:
            if file.endswith('.wav'):
                input_path = os.path.join(root, file)
                relative_path = os.path.relpath(root, input_dir)
                output_folder = os.path.join(output_dir, relative_path)
                load_chord(input_path, output_folder, unique_shapes)
    return unique_shapes

shapes_maj = process_chords('chords/maj', 'spectros/maj')
shapes_min = process_chords('chords/min', 'spectros/min')
print("Chords loaded and spectograms saved as images")
all_unique_shapes = shapes_maj.union(shapes_min)
print("Unique shapes of spectrogram images:", all_unique_shapes)

KeyboardInterrupt: 